In [1]:
import spektral
import tensorflow as tf
import numpy as np

print(tf.__version__)

2.6.0


Load and pre-process graph representation data.

In [14]:
adj, features, labels, train_mask, val_mask, test_mask = spektral.datasets.citation.load_data(dataset_name='cora')

Loading cora dataset
Pre-processing node features


In [15]:
features = features.todense()
# The adjacency matric doesn't come with self edges so we need to add the identity matrix 
adj = adj.todense() + np.eye(adj.shape[0])
features = features.astype('float32')
adj = adj.astype('float32')

print(features.shape[0], 'nodes')
print(features.shape[1], 'features in every node')
print(labels.shape[1], 'classes')

print(np.sum(train_mask), 'training nodes')
print(np.sum(val_mask), 'validation nodes')
print(np.sum(test_mask), 'test nodes')

2708 nodes
1433 features in every node
7 classes
140 training nodes
500 validation nodes
1000 test nodes


In [18]:
def masked_softmax_cross_entropy(logits, labels, mask):
    '''Applies loss function taking into account the mask to
       only take into account relevant nodes.
       
       Returns cross entropy loss over the masked nodes of the graph.'''
    loss = tf.nn.softmax_cross_entropy_with_logits(logits, labels)
    mask = tf.cast(mask, dtype=tf.float32)
    # Divide mask by its average value that will enable us to take the product of the mask with the loss
    mask /= tf.reduce_mean(mask)
    loss *= mask
    # return average across all positions
    return tf.reduce_mean(loss)


def masked_accuracy(logits, labels, mask):
    '''Returns accuracy over the masked nodes of the graph.'''
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy_all = tf.cast(correct_prediction, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    accuracy_all *= mask
    return tf.reduce_mean(accuracy_all)

In [ ]:
def gnn(fts, adj, tranform, activation)
    '''
    Define a Graph Neural Network layer.
    fts: node feature matrix
    adj: adjacency matrix
    transform: some transformation that we wish to apply to each node
    activation: activation function
    '''
    # Transform each node 
    seq_fts = transform(fts)
    # Once we have the features we want to aggregate we multiply by the adjacency matrix
    